# 🗂️ SQLite SQL — A Comprehensive Guide for Learners and Developers

*A Deep Dive into the SQL Language of the SQLite Engine*

---

## 🧭 Introduction

**SQLite** is a lightweight, server-less, self-contained SQL database engine that stores all its data in a single file. It implements most of the SQL-92 standard and is used in everything from web browsers and mobile apps to embedded systems and operating-system utilities.

Unlike server databases (SQL Server, MySQL, PostgreSQL), SQLite runs **in-process** with the application — there’s no external service to install or manage. You simply open a file, execute SQL statements, and the engine handles data persistence, indexing, and transactions automatically.

This guide provides a comprehensive overview of **SQLite’s SQL dialect**, its data model, and its unique implementation details. All examples use clean SQL syntax with explanatory commentary below each code block.

## ⚙️ Understanding SQLite Architecture

SQLite is a **file-based relational database**.

When you connect to a database such as `company.db`, SQLite creates (or opens) a single file on disk containing:

* Tables (data)
* Indexes
* Views
* Triggers
* The database schema

There’s no user authentication, no server configuration, and no networking — the database engine runs entirely in your application process.

SQLite supports both **persistent databases** (`.db` files) and **in-memory databases** (`:memory:`) that exist only during program execution.

## 🧱 Basic SQL Structure

The foundation of SQL in SQLite (and all relational systems) is built around four main statement categories:

| Category | Purpose | Examples |
|-----------|----------|-----------|
| **DDL (Data Definition Language)** | Create or alter database objects. | `CREATE TABLE`, `DROP TABLE`, `CREATE INDEX` |
| **DML (Data Manipulation Language)** | Insert, update, or delete data. | `INSERT`, `UPDATE`, `DELETE` |
| **DQL (Data Query Language)** | Retrieve data. | `SELECT`, `WITH`, `JOIN` |
| **DCL/Transaction Control** | Manage changes. | `BEGIN`, `COMMIT`, `ROLLBACK` |

Each section below explains these categories in practical detail.

## 📚 Creating Tables and Schemas

Tables store data as rows and columns, similar to spreadsheets but strongly typed and indexed.

In [ ]:
CREATE TABLE Employees (
    EmployeeID   INTEGER PRIMARY KEY,
    FirstName    TEXT NOT NULL,
    LastName     TEXT NOT NULL,
    Department   TEXT,
    HireDate     TEXT DEFAULT CURRENT_DATE,
    Salary       REAL
);

**Explanation:**

* `INTEGER PRIMARY KEY` defines a unique identifier. In SQLite, this automatically creates an **alias for the internal rowid** (a 64-bit integer unique to each record).
* `TEXT`, `REAL`, and `INTEGER` are **type affinities**, not rigid types — SQLite stores values dynamically while maintaining data consistency.
* `DEFAULT CURRENT_DATE` automatically inserts the current date on record creation.

### Viewing Tables

In [ ]:
.tables

In [ ]:
.schema Employees

## ✍️ Inserting Data

In [ ]:
INSERT INTO Employees (FirstName, LastName, Department, Salary)
VALUES ('Jane', 'Doe', 'Finance', 85000);

In [ ]:
INSERT INTO Employees (FirstName, LastName, Department, Salary)
VALUES
  ('John', 'Smith', 'HR', 72000),
  ('Alice', 'Brown', 'IT', 95000),
  ('Bob', 'Miller', 'Finance', 78000);

SQLite enforces constraints (`NOT NULL`, `UNIQUE`, `PRIMARY KEY`) automatically.

---

## 🔍 Querying Data with SELECT

In [ ]:
SELECT FirstName, LastName, Department
FROM Employees
WHERE Department = 'Finance'
ORDER BY LastName;

In [ ]:
SELECT FirstName || ' ' || LastName AS FullName, Salary AS AnnualPay
FROM Employees;

## ⚖️ Filtering Data with WHERE

Use logical and comparison operators to refine results.

| Operator | Meaning | Example |
|-----------|----------|----------|
| `=` | Equal to | `WHERE Department = 'IT'` |
| `<>` or `!=` | Not equal | `WHERE Department <> 'Finance'` |
| `<`, `>`, `<=`, `>=` | Comparison | `WHERE Salary >= 90000` |
| `AND`, `OR`, `NOT` | Combine conditions | `WHERE Department='IT' AND Salary>80000` |

In [ ]:
SELECT * FROM Employees WHERE Department IS NULL;

## 🔢 Sorting and Limiting Results

In [ ]:
SELECT * FROM Employees
ORDER BY Salary DESC;

In [ ]:
SELECT * FROM Employees
ORDER BY EmployeeID
LIMIT 5 OFFSET 10;

## 🧠 Understanding SQL Execution Order in SQLite

SQL queries are **not executed in the same order they’re written**. SQLite follows a **logical processing order** that determines how clauses are evaluated.

```
FROM → WHERE → GROUP BY → HAVING → SELECT → ORDER BY → LIMIT
```

## 🔗 Joining Tables

In [ ]:
SELECT e.FirstName, e.LastName, d.DepartmentName
FROM Employees AS e
INNER JOIN Departments AS d
ON e.Department = d.DepartmentName;

In [ ]:
SELECT c.CustomerName, o.OrderID
FROM Customers AS c
LEFT JOIN Orders AS o
ON c.CustomerID = o.CustomerID;

## 🧮 Aggregate Functions and Grouping

In [ ]:
SELECT Department, AVG(Salary) AS AvgSalary
FROM Employees
GROUP BY Department
HAVING AVG(Salary) > 80000;

## 🧩 Subqueries

In [ ]:
SELECT FirstName, LastName
FROM Employees
WHERE Department IN (
    SELECT DepartmentName FROM Departments WHERE Location = 'HQ'
);

In [ ]:
SELECT c.CustomerName
FROM Customers AS c
WHERE EXISTS (
    SELECT 1 FROM Orders AS o WHERE o.CustomerID = c.CustomerID
);

## ⚙️ Updating and Deleting Data

In [ ]:
UPDATE Employees
SET Salary = Salary * 1.10
WHERE Department = 'Sales';

In [ ]:
DELETE FROM Employees
WHERE HireDate < '2020-01-01';

## 🧱 Indexes and Keys

In [ ]:
CREATE INDEX idx_employees_department
ON Employees (Department);

In [ ]:
DROP INDEX idx_employees_department;

### Enabling Foreign Keys

In [ ]:
PRAGMA foreign_keys = ON;

## 🧾 Views

In [ ]:
CREATE VIEW vw_HighEarners AS
SELECT FirstName, LastName, Department, Salary
FROM Employees
WHERE Salary > 90000;

In [ ]:
SELECT * FROM vw_HighEarners;

In [ ]:
DROP VIEW vw_HighEarners;

## 🔄 Transactions

In [ ]:
BEGIN TRANSACTION;
UPDATE Accounts SET Balance = Balance - 500 WHERE AccountID = 1;
UPDATE Accounts SET Balance = Balance + 500 WHERE AccountID = 2;
COMMIT;

## 🧩 Triggers

In [ ]:
CREATE TRIGGER trg_UpdateAudit
AFTER UPDATE ON Employees
FOR EACH ROW
BEGIN
    INSERT INTO AuditLog (Action, TableName, RecordID, Timestamp)
    VALUES ('UPDATE', 'Employees', OLD.EmployeeID, datetime('now'));
END;

In [ ]:
DROP TRIGGER trg_UpdateAudit;

## 🧰 Built-In SQLite Functions

| Category | Function | Example | Description |
|-----------|-----------|----------|--------------|
| **String** | `LOWER(text)` | `UPPER(Name)` | Converts to uppercase |
| **Date/Time** | `date('now')` | | Current date |
| **Math** | `ABS(x)` | | Absolute value |
| **Aggregate** | `GROUP_CONCAT(expr, sep)` | | Concatenate group values |

## 🧮 The WITH Clause (CTEs)

In [ ]:
WITH DeptAverage AS (
    SELECT Department, AVG(Salary) AS AvgSalary
    FROM Employees
    GROUP BY Department
)
SELECT e.FirstName, e.LastName, e.Salary, d.AvgSalary
FROM Employees AS e
JOIN DeptAverage AS d ON e.Department = d.Department
WHERE e.Salary > d.AvgSalary;

## 📊 The PRAGMA Command

In [ ]:
PRAGMA foreign_keys = ON;
PRAGMA table_info(Employees);
PRAGMA database_list;
PRAGMA encoding;

## ⚡ Backup, Export, and Import

In [ ]:
.mode csv
.output employees.csv
SELECT * FROM Employees;
.output stdout

In [ ]:
.mode csv
.import employees.csv Employees

## ⚠️ Common Pitfalls and Best Practices

| Issue | Recommendation |
|--------|----------------|
| **Dynamic Typing** | Use `CHECK` constraints for strict validation. |
| **Foreign Keys Off by Default** | Always enable `PRAGMA foreign_keys=ON;`. |
| **Date Handling** | Dates stored as text; use `strftime()`. |
| **Transactions** | Use explicit transactions for performance. |
| **NULLs in Aggregates** | Use `COALESCE()` to replace NULLs. |
| **Case Sensitivity** | Use `COLLATE BINARY` for strict matching. |

## 🧩 Summary

SQLite’s SQL implementation is both **compact and powerful**.

Key takeaways:

* SQLite is **serverless** — everything lives in one file.
* It uses **dynamic typing** and **type affinities**.
* Supports **transactions, joins, triggers, and views**.
* Ideal for local storage and embedded systems.